# Moneyball

The purpose of this project is to build a team of nine players (one at each position) with the most value, using historical baseball statistics, using each'on-base %' coupled with.

This was the method used by the Oakland A's to build a championship calliber team on a shostring budget. The story was chronicled the book 'Moneyball'  and immortalized in the motion picture "Moneyball" starring Brad Pitt and Jonah Hill.

To accomplish this we will need each players 'On Base Percentage' and salary for each season played. 

### On Base Percentage (OBP)

Per Wikipedia: On-base percentage (OBP), also known as on-base average/OBA, measures how frequently a batter reaches base.[1] It is the ratio of the batter's times-on-base (TOB) (the sum of hits, walks, and times hit by pitch) to their number of plate appearances.[1] OBP does not credit the batter for reaching base due to fielding error, fielder's choice, dropped/uncaught third strike, fielder's obstruction, or catcher's interference.

#### The Formula

       OBP = (H + BB + HBP) / (AB + BB + HPB + SF)
      
Where:
* H = Hits
* BB = Bases on Balls (Walks)
* HBP = Hit By Pitch
* AB = At bat
* SF = Sacrifice fly

data courtesy of http://www.seanlahman.com/baseball-archive/statistics/

In [1]:
import pandas as pd
import numpy as np

In [2]:
# create the datetime objects on read
people_df = pd.read_csv('Data/People.csv',  parse_dates=['debut', 'finalGame'])
batting_df = pd.read_csv('Data/Batting.csv')

In [3]:
pd.set_option('display.max_columns', 45)
pd.set_option('display.max_rows', 90)

In [4]:
people_df.head(3)

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,Hank,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01


In [5]:
players = people_df[['playerID', 'nameFirst', 'nameLast', 'nameGiven']]

In [6]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19953 entries, 0 to 19952
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   playerID   19953 non-null  object
 1   nameFirst  19916 non-null  object
 2   nameLast   19953 non-null  object
 3   nameGiven  19916 non-null  object
dtypes: object(4)
memory usage: 623.7+ KB


In [7]:
batting_df.head(3)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,0,0,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,0,0,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,5,0,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0


In [8]:
batting = batting_df[['playerID', 'yearID', 'teamID', 'AB', 'H', 'BB', 'HBP', 'SF']]

In [9]:
batting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107429 entries, 0 to 107428
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   playerID  107429 non-null  object 
 1   yearID    107429 non-null  int64  
 2   teamID    107429 non-null  object 
 3   AB        107429 non-null  int64  
 4   H         107429 non-null  int64  
 5   BB        107429 non-null  int64  
 6   HBP       104612 non-null  float64
 7   SF        71325 non-null   float64
dtypes: float64(2), int64(4), object(2)
memory usage: 6.6+ MB


In [10]:
player_bb = pd.merge(players, batting)
len(player_bb)

107429

In [11]:
# checkpoint
player_bb1 = player_bb.copy()
player_bb1.fillna(0, inplace=True)
print(len(player_bb1))
player_bb1.head()

107429


,playerID,nameFirst,nameLast,nameGiven,yearID,teamID,AB,H,BB,HBP,SF
0,aardsda01,David,Aardsma,David Allan,2004,SFN,0,0,0,0.0,0.0
1,aardsda01,David,Aardsma,David Allan,2006,CHN,2,0,0,0.0,0.0
2,aardsda01,David,Aardsma,David Allan,2007,CHA,0,0,0,0.0,0.0
3,aardsda01,David,Aardsma,David Allan,2008,BOS,1,0,0,0.0,0.0
4,aardsda01,David,Aardsma,David Allan,2009,SEA,0,0,0,0.0,0.0


In [12]:
player_bb1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107429 entries, 0 to 107428
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   playerID   107429 non-null  object 
 1   nameFirst  107429 non-null  object 
 2   nameLast   107429 non-null  object 
 3   nameGiven  107429 non-null  object 
 4   yearID     107429 non-null  int64  
 5   teamID     107429 non-null  object 
 6   AB         107429 non-null  int64  
 7   H          107429 non-null  int64  
 8   BB         107429 non-null  int64  
 9   HBP        107429 non-null  float64
 10  SF         107429 non-null  float64
dtypes: float64(2), int64(4), object(5)
memory usage: 9.8+ MB


### Calculating 'On Base Percentage'

reminder: OBP = (H + BB + HBP) / (AB + BB + HPB + SF)

Remove outliers of less than 20 'At-Bats' to eliminate luck.

In [13]:
player_bb1.drop(player_bb1[player_bb1.AB < 20].index, inplace=True)

In [14]:
# Checkpoint
obp_df = player_bb1.copy()

In [15]:
obp_df['OBP'] = obp_df.apply(lambda row: (row['H']+row['BB']+row['HBP']) / (row['AB']+row['BB']+row['HBP']+row['SF']), axis='columns')
obp_df.sort_values(by=['OBP'], inplace=True, ascending=False)

In [16]:
obp_df.head()

,playerID,nameFirst,nameLast,nameGiven,yearID,teamID,AB,H,BB,HBP,SF,OBP
8824,bondsba01,Barry,Bonds,Barry Lamar,2004,SFN,373,135,232,9.0,3.0,0.609400
8822,bondsba01,Barry,Bonds,Barry Lamar,2002,SFN,403,149,198,9.0,2.0,0.581699
20442,craneed01,Ed,Crane,Edward Nicholas,1893,NY1,26,12,7,0.0,0.0,0.575758
68770,murphed02,Eddie,Murphy,John Edward,1919,CHA,35,17,7,0.0,0.0,0.571429
89055,sigmatr01,Tripp,Sigman,Wesley Triplett,1929,PHI,29,15,3,0.0,0.0,0.562500


### Player Value

Incorporate salary to determine a player's OBP value. (obp / salary)

In [17]:
salaries_df = pd.read_csv('Data/Salaries.csv')
salaries_df.head()

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000


In [18]:
salaries = salaries_df[['yearID', 'playerID', 'salary']]
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26428 entries, 0 to 26427
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   yearID    26428 non-null  int64 
 1   playerID  26428 non-null  object
 2   salary    26428 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 619.5+ KB


In [19]:
obp_w_salary = pd.merge(obp_df, salaries)
obp_w_salary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16759 entries, 0 to 16758
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   playerID   16759 non-null  object 
 1   nameFirst  16759 non-null  object 
 2   nameLast   16759 non-null  object 
 3   nameGiven  16759 non-null  object 
 4   yearID     16759 non-null  int64  
 5   teamID     16759 non-null  object 
 6   AB         16759 non-null  int64  
 7   H          16759 non-null  int64  
 8   BB         16759 non-null  int64  
 9   HBP        16759 non-null  float64
 10  SF         16759 non-null  float64
 11  OBP        16759 non-null  float64
 12  salary     16759 non-null  int64  
dtypes: float64(3), int64(5), object(5)
memory usage: 1.8+ MB


In [20]:
# Salaries of 0 will break calculation
obp_w_salary[obp_w_salary['salary'] <= 0]

,playerID,nameFirst,nameLast,nameGiven,yearID,teamID,AB,H,BB,HBP,SF,OBP,salary
1134,jamesdi01,Dion,James,Dion,1993,NYA,343,114,31,2.0,1.0,0.38992,0


In [21]:
obp_w_salary.drop(obp_w_salary[obp_w_salary['salary'] == 0].index , inplace=True)

In [22]:
# checkpoint
player_value = obp_w_salary.copy()
player_value.head(2)

,playerID,nameFirst,nameLast,nameGiven,yearID,teamID,AB,H,BB,HBP,SF,OBP,salary
0,bondsba01,Barry,Bonds,Barry Lamar,2004,SFN,373,135,232,9.0,3.0,0.609400,18000000
1,bondsba01,Barry,Bonds,Barry Lamar,2002,SFN,403,149,198,9.0,2.0,0.581699,15000000


In [23]:
player_value['OBP_to_salary'] = player_value.apply(lambda row: row['OBP'] / (row['salary'] / 100000), axis='columns')
player_value.drop(player_value[player_value['OBP_to_salary'] >= 1].index , inplace=True)
player_value.sort_values(by=['OBP_to_salary'], inplace=True, ascending=False)

In [25]:
player_value.head()

,playerID,nameFirst,nameLast,nameGiven,yearID,teamID,AB,H,BB,HBP,SF,OBP,salary,OBP_to_salary
366,mazzile01,Lee,Mazzilli,Lee Louis,1986,NYN,58,16,12,2.0,0.0,0.416667,60000,0.694444
589,jonestr01,Tracy,Jones,Tracy Donald,1986,CIN,86,30,9,0.0,1.0,0.406250,60000,0.677083
661,krukjo01,John,Kruk,John Martin,1986,SDN,278,86,45,0.0,2.0,0.403077,60000,0.671795
752,greenmi01,Mike,Greenwell,Michael Lewis,1986,BOS,35,11,5,0.0,0.0,0.400000,60000,0.666667
812,danieka01,Kal,Daniels,Kalvoski,1986,CIN,181,58,22,2.0,1.0,0.398058,60000,0.663430


### Player Postitions

Add data showing the postions each player had made appearances at. 

Postitions are pitcher, catcher, 1st base, 2nd base, short stop, 3rd base, left field, center field, right field

In [29]:
appearances_df = pd.read_csv('Data/Appearances.csv')
appearances_df.head()

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
0,1871,TRO,NaN,abercda01,1,1.0,1,1.0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0
1,1871,RC1,NaN,addybo01,25,25.0,25,25.0,0,0,0,22,0,3,0,0,0,0,0.0,0.0,0.0
2,1871,CL1,NaN,allisar01,29,29.0,29,29.0,0,0,0,2,0,0,0,29,0,29,0.0,0.0,0.0
3,1871,WS3,NaN,allisdo01,27,27.0,27,27.0,0,27,0,0,0,0,0,0,0,0,0.0,0.0,0.0
4,1871,RC1,NaN,ansonca01,25,25.0,25,25.0,0,5,1,2,20,0,1,0,0,1,0.0,0.0,0.0


In [31]:
appearances_df.columns

Index(['yearID', 'teamID', 'lgID', 'playerID', 'G_all', 'GS', 'G_batting',
       'G_defense', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf',
       'G_cf', 'G_rf', 'G_of', 'G_dh', 'G_ph', 'G_pr'],
      dtype='object')

In [38]:
col_list = ['yearID', 'playerID', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf',
       'G_cf', 'G_rf', 'G_dh']
appearances = appearances_df[col_list]
appearances.head()

,yearID,playerID,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_dh
0,1871,abercda01,0,0,0,0,0,1,0,0,0,0.0
1,1871,addybo01,0,0,0,22,0,3,0,0,0,0.0
2,1871,allisar01,0,0,0,2,0,0,0,29,0,0.0
3,1871,allisdo01,0,27,0,0,0,0,0,0,0,0.0
4,1871,ansonca01,0,5,1,2,20,0,1,0,0,0.0


In [34]:
value_w_pos = pd.merge(appearances, player_value)
value_w_pos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19514 entries, 0 to 19513
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   yearID         19514 non-null  int64  
 1   playerID       19514 non-null  object 
 2   G_p            19514 non-null  int64  
 3   G_c            19514 non-null  int64  
 4   G_1b           19514 non-null  int64  
 5   G_2b           19514 non-null  int64  
 6   G_3b           19514 non-null  int64  
 7   G_ss           19514 non-null  int64  
 8   G_lf           19514 non-null  int64  
 9   G_cf           19514 non-null  int64  
 10  G_rf           19514 non-null  int64  
 11  G_dh           19514 non-null  float64
 12  nameFirst      19514 non-null  object 
 13  nameLast       19514 non-null  object 
 14  nameGiven      19514 non-null  object 
 15  teamID         19514 non-null  object 
 16  AB             19514 non-null  int64  
 17  H              19514 non-null  int64  
 18  BB    

In [41]:
moneyball_df = value_w_pos.copy()
moneyball_df.to_csv('Data/moneyball_stats.csv')